In [28]:
import requests
import datetime
import pandas as pd
import requests
import hopsworks
from pathlib import Path
import json
import re
import os
import warnings
from dotenv import load_dotenv

In [29]:
# Load environment variables from .env file
load_dotenv()

HW_API_KEY = os.getenv("HOPSWORKS_API_KEY")
HW_PROJECT = os.getenv("HOPSWORKS_PROJECT")

if not HW_API_KEY or not HW_PROJECT:
    raise ValueError("Missing credentials in .env file")


# Connect to Hopsworks project and feature store
project = hopsworks.login(
    project=HW_PROJECT,
    api_key_value=HW_API_KEY,
    host="eu-west.cloud.hopsworks.ai",
)
fs = project.get_feature_store()

2026-01-11 16:21:20,422 INFO: Closing external client and cleaning up certificates.
2026-01-11 16:21:20,424 INFO: Connection closed.
2026-01-11 16:21:20,424 INFO: Initializing external client
2026-01-11 16:21:20,425 INFO: Base URL: https://eu-west.cloud.hopsworks.ai:443
2026-01-11 16:21:21,702 INFO: Python Engine initialized.

Logged in to project, explore it here https://eu-west.cloud.hopsworks.ai:443/p/2186


In [38]:
def get_prices(date, price_class):
    # url = (
    #     f"https://www.elprisetjustnu.se/api/v1/prices/"
    #     f"{date.year}/{date.month:02d}-{date.day:02d}_{price_class}.json"
    # )
    # r = requests.get(url)
    # r.raise_for_status()
    # return r.json()
    pass


In [31]:
today = datetime.date.today()
start = datetime.date(2021, 11, 1)
end   = today

current = start
all_data = []

while current <= end:
    try:
        daily_data = get_prices(current, "SE3")
        all_data.extend(daily_data)
    except requests.HTTPError:
        pass  # if any missing dates
    current += datetime.timedelta(days=1)

TypeError: 'NoneType' object is not iterable

In [44]:
#read csv file into dataframe
electricity_df = pd.read_csv("data/price_data_se3.csv")

In [39]:
electricity_df = pd.DataFrame(all_data)

In [ ]:
# write df to csv
# electricity_df.to_csv("data/price_data_se3.csv", index=False)

In [ ]:
# Timestamps to utc
time_start_utc = pd.to_datetime(electricity_df["time_start"], utc=True)
time_end_utc   = pd.to_datetime(electricity_df["time_end"], utc=True)

# Compute resolution in minutes (safe across DST)
electricity_df["resolution_minutes"] = (
    (time_end_utc - time_start_utc)
    .dt.total_seconds()
    .astype(int) // 60
)

# Use time_start as event time
electricity_df["date"] = time_start_utc

# Drop original time columns
electricity_df = electricity_df.drop(columns=["time_start", "time_end"])
# Add price area
electricity_df["price_area"] = "SE3"

# Reorder (optional)
electricity_df = electricity_df[
    ["date", "resolution_minutes",
     "SEK_per_kWh", "EUR_per_kWh", "EXR", "price_area"]
]

print(electricity_df.dtypes)
print(electricity_df.head())

date                  datetime64[ns, UTC]
resolution_minutes                  int64
SEK_per_kWh                       float64
EUR_per_kWh                       float64
EXR                               float64
price_area                         object
dtype: object
                       date  resolution_minutes  SEK_per_kWh  EUR_per_kWh  \
0 2026-01-06 23:00:00+00:00                  15      0.80725      0.07500   
1 2026-01-06 23:15:00+00:00                  15      0.85924      0.07983   
2 2026-01-06 23:30:00+00:00                  15      0.78798      0.07321   
3 2026-01-06 23:45:00+00:00                  15      0.72652      0.06750   
4 2026-01-07 00:00:00+00:00                  15      0.78486      0.07292   

         EXR price_area  
0  10.763315        SE3  
1  10.763315        SE3  
2  10.763315        SE3  
3  10.763315        SE3  
4  10.763315        SE3  


In [42]:
# Get or create feature group 
electricity_fg = fs.get_or_create_feature_group(
    name='electricity',
    description='Electricity prices for each day',
    version=1,
    primary_key=['price_area'],
    event_time="date",
)

In [43]:
# Insert data
electricity_fg.insert(electricity_df, wait=True)

2026-01-11 16:29:02,641 INFO: Computing insert statistics


(None, None)

In [ ]:
electricity_fg.update_feature_description("date", "Timestamp (UTC) of measurement of electricity price")
electricity_fg.update_feature_description("resolution_minutes", "Resolution of the price measurement in minutes")
electricity_fg.update_feature_description("sek_per_kwh", "Electricity price in SEK per kWh")
electricity_fg.update_feature_description("eur_per_kwh", "Electricity price in EUR per kWh")
electricity_fg.update_feature_description("exr", "Exchange rate used when conversion from EUR to SEK was done")
electricity_fg.update_feature_description("price_area", "Price area where electricity price is measured")